In [2]:
### Config
### Import dependencies
# Dependencies for data manipulation
import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from dateutil.relativedelta import *

# Dependencies for Databases
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
from sqlalchemy import create_engine

# Find environment variables
DATABASE_URL = os.environ.get("DATABASE_URL", None)
# sqlalchemy deprecated urls which begin with "postgres://"; now it needs to start with "postgresql://"
if DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)

In [79]:
### Build Postgresql table for historical player stats
# Get historical data
histdf = pd.read_csv('onetimeData/modelSource.csv', index_col=0)

# Select only previous two years
histdf = histdf.loc[histdf['season']>=2020]

# Clean defense names in historical data
histdf['player'] = histdf['player'].str.upper()
# Drop punctuation
histdf['player'] = histdf['player'].str.replace(".", "")
histdf['player'] = histdf['player'].str.replace(",", "")
histdf['player'] = histdf['player'].str.replace("'", "")

histdf = histdf[[
    'season', 'week', 
    'team', 'opponent',
    'player', 'pos', 'posRank', 
    'passA', 'passC', 'passY', 'passT', 'passI', 'pass2', 
    'rushA', 'rushY', 'rushT', 'rush2', 
    'recC', 'recY', 'recT', 'rec2', 'fum', 
    'XPA', 'XPM', 'FGA', 'FGM', 'FG50', 
    'defSack', 'defI', 'defSaf', 'defFum', 'defBlk', 'defT', 'defPtsAgainst', 'defPassYAgainst', 'defRushYAgainst', 'defYdsAgainst']]


/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # This is added back by InteractiveShellApp.init_path()


In [80]:

# Save space by creating a yearly summary for historical data
## histdf = histdf.loc[histdf.season>=2020]
# histdf_avg = histdf.groupby(['season', 'player']).mean()
# histdf_avg['gamesPlayed'] = histdf.groupby(['season', 'player'])['week'].count()
# histdf_avg.drop(columns='week', inplace=True)
# gamesPlayedCol = histdf_avg.pop('gamesPlayed')
# histdf_avg.insert(0, 'gamesPlayed', gamesPlayedCol)
# histdf = histdf_avg
# histdf.reset_index(inplace=True)


In [91]:
# Find average stats for 1 season prior
prior1 = histdf.loc[histdf.season==2021]
prior1_avg = prior1.groupby(['player']).mean()
prior1_avg['gamesPlayed'] = prior1.groupby(['player'])['week'].count()
prior1_avg.drop(columns=['week', 'season'], inplace=True)
gamesPlayedCol = prior1_avg.pop('gamesPlayed')
prior1_avg.insert(0, 'gamesPlayed', gamesPlayedCol)
prior1 = prior1_avg
prior1.reset_index(inplace=True)
# Rename all columns in prior1
colList = [(x + "_prior1") for x in list(prior1.columns)]
prior1.columns = colList
prior1 = prior1.rename(columns={'player_prior1':'player'})
prior1


,player,gamesPlayed_prior1,passA_prior1,passC_prior1,passY_prior1,passT_prior1,passI_prior1,pass2_prior1,rushA_prior1,rushY_prior1,...,defSack_prior1,defI_prior1,defSaf_prior1,defFum_prior1,defBlk_prior1,defT_prior1,defPtsAgainst_prior1,defPassYAgainst_prior1,defRushYAgainst_prior1,defYdsAgainst_prior1
0,AARON JONES,15,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,11.400000,53.266667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AARON RODGERS,15,34.20,23.466667,265.133333,2.333333,0.266667,0.000000,2.133333,6.266667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ADAM HUMPHRIES,16,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ADAM PRENTICE,3,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,1.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ADAM SHAHEEN,6,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,ZACH PASCAL,14,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,1.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
661,ZACH WILSON,12,30.25,17.166667,187.250000,0.666667,0.916667,0.083333,2.250000,13.416667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
662,ZACK MOSS,12,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,7.583333,28.083333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
663,ZANE GONZALEZ,12,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [109]:
# Find average stats for 2 seasons prior
prior2 = histdf.loc[histdf.season>=2020]
prior2_avg = prior2.groupby(['player']).mean()
prior2_avg['gamesPlayed'] = prior2.groupby(['player'])['week'].count()
prior2_avg.drop(columns=['week', 'season'], inplace=True)
gamesPlayedCol = prior2_avg.pop('gamesPlayed')
prior2_avg.insert(0, 'gamesPlayed', gamesPlayedCol)
prior2 = prior2_avg
prior2.reset_index(inplace=True)
# Rename all columns in prior2
colList = [(x + "_prior2") for x in list(prior2.columns)]
prior2.columns = colList
prior2 = prior2.rename(columns={'player_prior2':'player'})
prior2

,player,gamesPlayed_prior2,passA_prior2,passC_prior2,passY_prior2,passT_prior2,passI_prior2,pass2_prior2,rushA_prior2,rushY_prior2,...,defSack_prior2,defI_prior2,defSaf_prior2,defFum_prior2,defBlk_prior2,defT_prior2,defPtsAgainst_prior2,defPassYAgainst_prior2,defRushYAgainst_prior2,defYdsAgainst_prior2
0,AARON JONES,29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.827586,65.620690,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AARON RODGERS,31,33.516129,23.354839,266.967742,2.677419,0.290323,0.000000,2.258065,7.838710,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ADAM HUMPHRIES,22,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ADAM PRENTICE,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,1.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ADAM SHAHEEN,15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817,ZACH PASCAL,30,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.700000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
818,ZACH WILSON,12,30.250000,17.166667,187.250000,0.666667,0.916667,0.083333,2.250000,13.416667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
819,ZACK MOSS,25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.120000,32.720000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
820,ZANE GONZALEZ,24,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
# Send dfs to database
# Prepare prior1 df
# Build the SQL query that will list columns and datatypes
string1 = [x + " VARCHAR(32)" for x in prior1.columns[:1]] + [
    x + " SMALLINT" for x in prior1.columns[1:2]] + [
    x + " FLOAT(8)" for x in prior1.columns[2:]] 
string1 = str(string1)
string1 = string1.replace("'", "")
string1 = string1.replace("[", "")
string1 = string1.replace("]", "")
#print(f'CREATE TABLE IF NOT EXISTS prior1({string1})')

# Write the df to the Postgresql database
try:
    # connect to database
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    engine = create_engine(DATABASE_URL)
    cursor = conn.cursor()
    # Create table for prior1
    cursor.execute(f'CREATE TABLE IF NOT EXISTS prior1({string1})')
    conn.commit()
    # Populate table with data
    prior1.to_sql('prior1', engine, if_exists='replace', index = False)
except Exception as error:
    print(error)
finally:
    if conn:
        cursor.close()
        conn.close()

In [110]:
# Send dfs to database
# Prepare prior2 df
# Build the SQL query that will list columns and datatypes
string2 = [x + " VARCHAR(32)" for x in prior2.columns[:1]] + [
    x + " SMALLINT" for x in prior2.columns[1:2]] + [
    x + " FLOAT(8)" for x in prior2.columns[2:]] 
string2 = str(string2)
string2 = string2.replace("'", "")
string2 = string2.replace("[", "")
string2 = string2.replace("]", "")
#print(f'CREATE TABLE IF NOT EXISTS prior2({string2})')

# Write the df to the Postgresql database
try:
    # connect to database
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    engine = create_engine(DATABASE_URL)
    cursor = conn.cursor()
    # Create table for prior2
    cursor.execute(f'CREATE TABLE IF NOT EXISTS prior2({string2})')
    conn.commit()
    # Populate table with data
    prior2.to_sql('prior2', engine, if_exists='replace', index = False)
except Exception as error:
    print(error)
finally:
    if conn:
        cursor.close()
        conn.close()

In [37]:
# Create empty current df
# colList = [(x + "_curr") for x in list(histdf.columns)]
# curr = pd.DataFrame(columns=colList)
# curr = curr.rename(columns={'player_curr':'player'})
# curr.drop(columns=['season_curr', 'week_curr', 'team_curr', 'opponent_curr', 'pos_curr', 'posRank_curr'], inplace=True)
# curr['player'] = histdf['player']
# curr.fillna(0, inplace=True)
# curr

In [103]:
# # Send dfs to database
# # Prepare curr df
# # Build the SQL query that will list columns and datatypes
# string1 = [x + " VARCHAR(32)" for x in curr.columns[:1]] + [
#     x + " SMALLINT" for x in curr.columns[1:2]] + [
#     x + " FLOAT(8)" for x in curr.columns[2:]] 
# string1 = str(string1)
# string1 = string1.replace("'", "")
# string1 = string1.replace("[", "")
# string1 = string1.replace("]", "")
# #print(f'CREATE TABLE IF NOT EXISTS curr({string1})')

# # Write the df to the Postgresql database
# try:
#     # connect to database
#     conn = psycopg2.connect(DATABASE_URL, sslmode='require')
#     engine = create_engine(DATABASE_URL)
#     cursor = conn.cursor()
#     # Create table for curr
#     cursor.execute(f'CREATE TABLE IF NOT EXISTS curr({string1})')
#     conn.commit()
#     # Populate table with data
#     curr.to_sql('curr', engine, if_exists='replace', index = False)
# except Exception as error:
#     print(error)
# finally:
#     if conn:
#         cursor.close()
#         conn.close()

In [107]:
# Send schedule to database
schedule = pd.read_csv('onetimeData/schedule.csv')
schedule['team'] = schedule['team'].str.upper()
schedule['opponent'] = schedule['opponent'].str.upper()

initialList = ['BUF', 'IND', 'MIA', 'NEP', 'NYJ', 'CIN', 'CLE', 'TEN', 'JAC',
       'PIT', 'DEN', 'KCC', 'LVR', 'LAC', 'SEA', 'DAL', 'NYG', 'PHI',
       'ARI', 'WAS', 'CHI', 'DET', 'GBP', 'MIN', 'TBB', 'ATL', 'CAR',
       'LAR', 'NOS', 'SFO', 'BAL', 'HOU']
teamList = ['BUFFALO BILLS', 'INDIANAPOLIS COLTS', 'MIAMI DOLPHINS',
       'NEW ENGLAND PATRIOTS', 'NEW YORK JETS', 'CINCINNATI BENGALS',
       'CLEVELAND BROWNS', 'TENNESSEE TITANS', 'JACKSONVILLE JAGUARS',
       'PITTSBURGH STEELERS', 'DENVER BRONCOS', 'KANSAS CITY CHIEFS',
       'LAS VEGAS RAIDERS', 'LOS ANGELES CHARGERS', 'SEATTLE SEAHAWKS',
       'DALLAS COWBOYS', 'NEW YORK GIANTS', 'PHILADELPHIA EAGLES',
       'ARIZONA CARDINALS', 'WASHINGTON COMMANDERS', 'CHICAGO BEARS',
       'DETROIT LIONS', 'GREEN BAY PACKERS', 'MINNESOTA VIKINGS',
       'TAMPA BAY BUCCANEERS', 'ATLANTA FALCONS', 'CAROLINA PANTHERS',
       'LOS ANGELES RAMS', 'NEW ORLEANS SAINTS', 'SAN FRANCISCO 49ERS',
       'BALTIMORE RAVENS', 'HOUSTON TEXANS']
initialsZipped = dict(zip(teamList, initialList))

schedule = schedule.replace({'team':initialsZipped})
schedule = schedule.replace({'opponent':initialsZipped})
schedule

,week,team,opponent
0,1,BUF,LAR
1,1,NOS,ATL
2,1,CLE,CAR
3,1,SFO,CHI
4,1,PIT,CIN
...,...,...,...
539,18,PIT,CLE
540,18,LVR,KCC
541,18,SEA,LAR
542,18,SFO,ARI


In [108]:
# Send schedule to database
# Prepare schedule df
# Build the SQL query that will list columns and datatypes
string1 = [x + " SMALLINT" for x in schedule.columns[:1]] + [
    x + " VARCHAR(32)" for x in schedule.columns[1:]]
string1 = str(string1)
string1 = string1.replace("'", "")
string1 = string1.replace("[", "")
string1 = string1.replace("]", "")
#print(f'CREATE TABLE IF NOT EXISTS schedule({string1})')

# Write the df to the Postgresql database
try:
    # connect to database
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    engine = create_engine(DATABASE_URL)
    cursor = conn.cursor()
    # Create table for schedule
    cursor.execute(f'CREATE TABLE IF NOT EXISTS schedule({string1})')
    conn.commit()
    # Populate table with data
    schedule.to_sql('schedule', engine, if_exists='replace', index = False)
except Exception as error:
    print(error)
finally:
    if conn:
        cursor.close()
        conn.close()

In [30]:
# Get table names
connection = False
try:
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    cursor = conn.cursor()
    query = f'SELECT table_name FROM information_schema.tables'
    cursor.execute(query)
    result = pd.read_sql(query, conn)
except (Exception) as error:
    print(error)
finally:
    if connection:
        cursor.close()
        connection.close()
str(list(result.table_name))

"['prior2', 'player_df', 'player_dobs', 'predictions', 'point_projections', 'pg_type', 'schedule', 'pg_foreign_table', 'prior1', 'pg_roles', 'pg_settings', 'pg_available_extension_versions', 'pg_stat_xact_user_functions', 'pg_subscription', 'pg_attribute', 'pg_proc', 'pg_class', 'pg_attrdef', 'pg_constraint', 'pg_inherits', 'pg_index', 'pg_operator', 'pg_opfamily', 'pg_opclass', 'pg_am', 'pg_amop', 'pg_amproc', 'pg_language', 'pg_stat_archiver', 'pg_stat_bgwriter', 'pg_stat_wal', 'pg_stat_progress_analyze', 'pg_stat_progress_vacuum', 'pg_stat_progress_cluster', 'pg_stat_progress_create_index', 'pg_stat_progress_basebackup', 'pg_stat_progress_copy', 'pg_largeobject_metadata', 'pg_aggregate', 'pg_statistic_ext', 'pg_rewrite', 'pg_trigger', 'pg_event_trigger', 'pg_description', 'pg_cast', 'pg_enum', 'pg_namespace', 'pg_conversion', 'pg_depend', 'pg_database', 'pg_db_role_setting', 'pg_tablespace', 'pg_auth_members', 'pg_shdepend', 'pg_shdescription', 'pg_ts_config', 'pg_ts_config_map', 'p

In [36]:
# Get predictions
connection = False
try:
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    cursor = conn.cursor()
    query = f'SELECT * FROM curr;'
    cursor.execute(query)
    result = pd.read_sql(query, conn)
except (Exception) as error:
    print(error)
finally:
    if connection:
        cursor.close()
        connection.close()
result

,player,passA_curr,passC_curr,passY_curr,passT_curr,passI_curr,pass2_curr,rushA_curr,rushY_curr,rushT_curr,...,defSack_curr,defI_curr,defSaf_curr,defFum_curr,defBlk_curr,defT_curr,defPtsAgainst_curr,defPassYAgainst_curr,defRushYAgainst_curr,defYdsAgainst_curr
0,AARON RODGERS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,RUSSELL WILSON,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,JOSH ALLEN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,LAMAR JACKSON,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MITCHELL TRUBISKY,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# Drop table
connection = False
try:
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    cursor = conn.cursor()
    query = f'DROP TABLE curr;'
    cursor.execute(query)
    #result = pd.read_sql(query, conn)
except (Exception) as error:
    print(error)
finally:
    if connection:
        cursor.close()
        connection.close()

In [35]:
stringCurr = [x + " VARCHAR(32)" for x in curr.columns[:1]] + [
    x + " FLOAT(8)" for x in curr.columns[2:]] 
stringCurr = str(stringCurr)
stringCurr = stringCurr.replace("'", "")
stringCurr = stringCurr.replace("[", "")
stringCurr = stringCurr.replace("]", "")


## Remove curr table from db
# Write the df to the Postgresql database
try:
    # connect to database
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    engine = create_engine(DATABASE_URL)
    cursor = conn.cursor()
    # Create table for schedule
    cursor.execute(f'CREATE TABLE IF NOT EXISTS curr({stringCurr})')
    conn.commit()
    # Populate table with data
    curr.to_sql('curr', engine, if_exists='replace', index = False)
except Exception as error:
    print(error)
finally:
    if conn:
        cursor.close()
        conn.close()

In [40]:
# Get predictions
connection = False
try:
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    cursor = conn.cursor()
    query = f'SELECT * FROM predictions;'
    cursor.execute(query)
    result = pd.read_sql(query, conn)
except (Exception) as error:
    print(error)
finally:
    if connection:
        cursor.close()
        connection.close()
result

,id_mfl,player,age,team,pos,posRank,KR,PR,RES,sharkRank,adp,pred,sharkRelative,sharkAbsolute,adpRelative,adpAbsolute
0,5848,TOM BRADY,45.0,TBB,QB,QB1,NO,NO,NO,68.0,60.97,347.798672,171.646664,329.246664,136.640001,294.240001
1,14779,JUSTIN HERBERT,24.0,LAC,QB,QB1,NO,NO,NO,60.0,30.74,341.323461,178.046668,335.646667,178.046668,335.646667
2,13116,PATRICK MAHOMES,26.0,KCC,QB,QB1,NO,NO,NO,55.0,27.14,339.347453,193.409999,351.009999,193.409999,351.009999
3,12620,DAK PRESCOTT,29.0,DAL,QB,QB1,NO,NO,NO,86.0,69.99,332.541423,136.640001,294.240001,118.080001,275.680001
4,9431,MATTHEW STAFFORD,34.0,LAR,QB,QB1,NO,NO,NO,123.0,83.62,331.415001,67.943333,225.543333,85.600002,243.200002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,14838,LAVISKA SHENAULT,23.0,CAR,WR,WR3,NO,NO,NO,671.0,3000.00,24.121850,-46.450000,69.150000,-96.230000,19.370000
621,14555,GUNNER OLSZEWSKI,25.0,PIT,WR,WR2,KR1,PR1,NO,1246.0,3000.00,107.820542,-84.293334,31.306666,-59.080001,56.519999
622,12394,DEANDRE CARTER,29.0,LAC,WR,WR2,KR1,PR1,NO,1031.0,3000.00,105.890985,-60.013334,55.586666,-96.823333,18.776667
623,14108,GREG DORTCH,24.0,ARI,WR,WR2,KR2,PR2,NO,3000.0,3000.00,21.435179,-92.880000,22.720000,-48.813333,66.786667


In [52]:
import requests
from bs4 import BeautifulSoup

user_league = "53906"
user_franchise = "0005"

# Get Franchises in the league
urlString = f"https://www54.myfantasyleague.com/2022/export?TYPE=league&L={user_league}"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')
data = []
franchises = soup.find_all('franchise')
for i in range(len(franchises)):
    rows = [franchises[i].get("id"), franchises[i].get("name")]
    data.append(rows)
franchise_df = pd.DataFrame(data)
franchise_df.columns=['FranchiseID','FranchiseName']
franchise_df = franchise_df.append({"FranchiseID":"FA", "FranchiseName":"Free Agent"}, ignore_index=True)

# Get franchise rosters
urlString = f"https://www54.myfantasyleague.com/2022/export?TYPE=rosters&L={user_league}"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')
data = []
franchises = soup.find_all('franchise')
for i in range(0,len(franchises)):
    current_franchise = franchises[i].find_all('player')
    for j in range(0,len(current_franchise)):
        rows = [franchises[i].get("id"), franchises[i].get("week"), current_franchise[j].get("id"), current_franchise[j].get("status")]
        data.append(rows)
rosters_df = pd.DataFrame(data)

# Get Free Agents
urlString = f"https://www54.myfantasyleague.com/2022/export?TYPE=freeAgents&L={user_league}"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')
data = []
freeAgents = soup.find_all('player')
for i in range(len(freeAgents)):
    rows = ["FA", "", freeAgents[i].get("id"), "Free Agent"]
    data.append(rows)
fa_df = pd.DataFrame(data)
rosters_df = rosters_df.append(fa_df)
rosters_df.columns=['FranchiseID','Week','PlayerID','RosterStatus']

# Get all players, sharkRank, and ADP
# Get predictions
connection = False
try:
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    cursor = conn.cursor()
    query = f'SELECT * FROM predictions;'
    cursor.execute(query)
    result = pd.read_sql(query, conn)
except (Exception) as error:
    print(error)
finally:
    if connection:
        cursor.close()
        connection.close()
result
player_df = result

# Merge all dfs
a = player_df.merge(rosters_df, left_on='id_mfl', how='left', right_on='PlayerID').merge(franchise_df[['FranchiseID', 'FranchiseName']], on='FranchiseID', how='left')
b = a[a['FranchiseID'].notna()]
c = b.sort_values(by=['pred'], ascending=False)
c.reset_index(inplace=True, drop=True)

In [55]:
c.to_csv('cutter.csv')